# Melaksanakan Sistem Pakar Haiwan

Contoh daripada [Kurikulum AI untuk Pemula](http://github.com/microsoft/ai-for-beginners).

Dalam contoh ini, kita akan melaksanakan sistem berasaskan pengetahuan yang ringkas untuk menentukan haiwan berdasarkan beberapa ciri fizikal. Sistem ini boleh diwakili oleh pokok AND-OR berikut (ini adalah sebahagian daripada keseluruhan pokok, kita boleh dengan mudah menambah beberapa peraturan lagi):

![](../../../../../../translated_images/ms/AND-OR-Tree.5592d2c70187f283.webp)


## Sistem pakar shell kami sendiri dengan inferens kebelakang

Mari cuba mentakrifkan bahasa ringkas untuk perwakilan pengetahuan berdasarkan peraturan pengeluaran. Kami akan menggunakan kelas Python sebagai kata kunci untuk menentukan peraturan. Pada dasarnya terdapat 3 jenis kelas:
* `Ask` mewakili soalan yang perlu ditanya kepada pengguna. Ia mengandungi set jawapan yang mungkin.
* `If` mewakili peraturan, dan ia hanyalah gula sintaks untuk menyimpan kandungan peraturan
* `AND`/`OR` adalah kelas untuk mewakili cabang AND/OR pokok. Mereka hanya menyimpan senarai hujah di dalamnya. Untuk memudahkan kod, semua fungsi ditakrifkan dalam kelas induk `Content`


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

Dalam sistem kami, memori kerja akan mengandungi senarai **fakta** sebagai **pasangan atribut-nilai**. Pangkalan pengetahuan boleh ditakrifkan sebagai satu kamus besar yang memetakan tindakan (fakta baru yang harus dimasukkan ke dalam memori kerja) kepada syarat, yang dinyatakan sebagai ungkapan AND-OR. Juga, sesetengah fakta boleh `Ditanya`.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Untuk melakukan inferens balik, kami akan mentakrifkan kelas `Knowledgebase`. Ia akan mengandungi:
* `memory` kerja - sebuah kamus yang memetakan atribut kepada nilai
* Peraturan `rules` Knowledgebase dalam format seperti yang ditakrifkan di atas

Dua kaedah utama adalah:
* `get` untuk mendapatkan nilai atribut, melaksanakan inferens jika perlu. Contohnya, `get('color')` akan mendapatkan nilai ruang warna (ia akan bertanya jika perlu, dan menyimpan nilai tersebut untuk digunakan kemudian dalam memory kerja). Jika kami bertanya `get('color:blue')`, ia akan bertanya untuk warna, dan kemudian mengembalikan nilai `y`/`n` bergantung pada warna.
* `eval` melaksanakan inferens sebenar, iaitu mengembara pokok AND/OR, menilai sub-matlamat, dan lain-lain.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Sekarang mari kita takrifkan pangkalan pengetahuan haiwan kita dan lakukan perundingan. Nota bahawa panggilan ini akan mengemukakan soalan kepada anda. Anda boleh menjawab dengan menaip `y`/`n` untuk soalan ya-tidak, atau dengan menyatakan nombor (0..N) untuk soalan dengan jawapan pilihan berganda yang lebih panjang.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Menggunakan Experta untuk Inferens Hadapan

Dalam contoh seterusnya, kita akan cuba melaksanakan inferens hadapan menggunakan salah satu perpustakaan untuk perwakilan pengetahuan, [Experta](https://github.com/nilp0inter/experta). **Experta** adalah perpustakaan untuk mencipta sistem inferens hadapan dalam Python, yang direka untuk serupa dengan sistem lama klasik [CLIPS](http://www.clipsrules.net/index.html). 

Kita juga boleh melaksanakan pendahanan hadapan sendiri tanpa banyak masalah, tetapi pelaksanaan naif biasanya tidak begitu cekap. Untuk pemadanan peraturan yang lebih berkesan, algoritma khas [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) digunakan.


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

Kita akan mentakrifkan sistem kita sebagai kelas yang mewarisi `KnowledgeEngine`. Setiap peraturan ditakrifkan oleh fungsi yang berasingan dengan anotasi `@Rule`, yang menentukan bila peraturan itu harus dijalankan. Di dalam peraturan, kita boleh menambah fakta baru menggunakan fungsi `declare`, dan penambahan fakta-fakta tersebut akan menyebabkan beberapa peraturan lain dipanggil oleh enjin inferens hadapan.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Setelah kita telah mentakrifkan asas pengetahuan, kita mengisi ingatan kerja kita dengan beberapa fakta awal, dan kemudian memanggil kaedah `run()` untuk melakukan inferens. Anda boleh lihat hasilnya fakta-fakta baru yang diperoleh ditambahkan ke dalam ingatan kerja, termasuk fakta akhir tentang haiwan tersebut (jika kita menetapkan semua fakta awal dengan betul).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asli harus dianggap sebagai sumber yang sahih. Untuk maklumat kritikal, terjemahan profesional oleh manusia adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
